In [ ]:
# if backing up to Google Drive is needed
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Required packages
!pip install telethon py7zr rarfile

In [ ]:
##########################
# Get Private Channel ID #
##########################

import asyncio
from telethon.sync import TelegramClient, events
from telethon.tl.types import PeerChannel

# Your Telegram API credentials
API_ID = ''
API_HASH = ''
SESSION_NAME = ''
client = None
received_channels = []
channel_lock = asyncio.Lock()

async def get_all_new_message_channel_id(event):
    global received_channels
    async with channel_lock:
      if event.peer_id.channel_id not in received_channels: #skip already captured channels
        try:
          result = await client.get_entity(PeerChannel(event.peer_id.channel_id))
        except Exception as e:
          print(e, " - ", event)
          return
        else:
          # Print the channel Title & ID
          print("Channel: ", result.title, ", Channel ID:", result.id)
          received_channels.append(result.id)



async def event_listener():
    global client
    # Create a Telethon client
    async with TelegramClient(SESSION_NAME, API_ID, API_HASH) as client:
      # Add event handler to filter and forward file messages
      client.add_event_handler(get_all_new_message_channel_id, event=events.NewMessage())

      print("Public & private channel names and IDs will be shown if any new messages are sent(either by you or anyone else) to them...")
      print("Wait until your prefered private channel shows up, then copy the Channel ID to 'Forward Message' section...")

      # Start the event listener
      await client.run_until_disconnected()


try:
  '''
  loop = asyncio.get_event_loop()
  loop.run_until_complete(event_listener())
  '''
  asyncio.run(event_listener())
except RuntimeError:
  await event_listener()

In [ ]:
###################
# Forward Message #
###################

import asyncio, os
from telethon.sync import TelegramClient, events
from telethon.tl.types import Message, MessageMediaDocument, MessageMediaPhoto, DocumentAttributeFilename, PeerChannel
from telethon.errors import MessageIdInvalidError
import zipfile, py7zr, rarfile

# Your Telegram API credentials
API_ID = ''
API_HASH = ''
SESSION_NAME = ''

# Channel links to forward & path for downloads
channels_to_listen = ["t.me/?????", 123456789]
DOWNLOAD_PATH = './telegram_download'

client = None




async def forward_to_saved_messages(event):
  #print(type(event))
  #print(event)

  '''
  #Foward text message
  if isinstance(event.message, Message):
    try:
      await client.send_message('me', event.message.message)
    except ValueError: #the message could be a channel group specific sticker or gif, it's fine
      print("❌ Cannot foward text message: ", event)
    else:
      print("✉️ Successfully foward text: ", event.message.message)
  '''


  # Foward file & video message
  if isinstance(event.media, MessageMediaDocument):
    try:
      #await client.forward_messages('me', event.message)
      message = await client.send_file('me', event.message.media.document)
      file_name = get_file_name(event)
    except ValueError:
      print("❌ Cannot foward file message: ", file_name)
    except MessageIdInvalidError: #the message no longer exists (e.g., it has been deleted or expired)
      print("❌ Cannot foward file message (id): ", file_name)
    else:
      print("📎 Successfully foward file: ", file_name)
      #await download_file(event.message)
      await download_forwarded_messages(message)



  # Foward photo message
  if isinstance(event.media, MessageMediaPhoto):
    try:
      #await client.forward_messages('me', event.message)
      message = await client.send_file('me', event.message.media.photo)
    except MessageIdInvalidError: #the message no longer exists (e.g., it has been deleted or expired)
      print("❌ Cannot foward file message (id) - ", event)
    else:
      print("🖼️ Successfully foward photo - ", event)
      #await download_photo(event.message)
      await download_forwarded_messages(message)





async def download_forwarded_messages(event):
    #print(event)
    #print(type(event))

    # Download forwarded file & video message
    if isinstance(event.media, MessageMediaDocument):
      try:
        await download_file(event)
      except Exception as e:
        print("❌ Cannot download fowarded file message: ", e, ' - ', event)


    # Download forwarded photo message
    if isinstance(event.media, MessageMediaPhoto):
      try:
        await download_photo(event)
      except Exception as e:
        print("❌ Cannot download fowarded photo message: ", e, ' - ', event)





def get_file_name(event):
    #prints file name & whole Raw message
    file_name = None
    for attribute in event.message.document.attributes:
      if isinstance(attribute, DocumentAttributeFilename):
        file_name = attribute.file_name
        break
    if file_name:
      return file_name + " - " + str(event)
    else:
      return "Cannot get file name - " + str(event)



def is_compressed_file(file_path):
    # Check if the downloaded file is a .zip, .7z, or .rar file
    extensions = ('.zip', '.7z', '.rar')
    return any(file_path.lower().endswith(ext) for ext in extensions)


def extract_compressed_file(file_path, extract_folder):
    # Extract the compressed file to a new folder with the same name as the compressed file
    extension = os.path.splitext(file_path)[1].lower()

    try:
      if extension == '.zip':
          with zipfile.ZipFile(file_path, 'r') as zip_ref:
              zip_ref.extractall(extract_folder)
          print("✅ Extracted zip file:", file_path)

      elif extension == '.7z':
          with py7zr.SevenZipFile(file_path, mode='r') as seven_zip:
              seven_zip.extractall(path=extract_folder)
          print("✅ Extracted 7z file:", file_path)

      elif extension == '.rar':
          with rarfile.RarFile(file_path, 'r') as rar_ref:
              rar_ref.extractall(extract_folder)
          print("✅ Extracted rar file:", file_path)

    except Exception as e:
      print("❌ Error while extracting file:", e, " - ", file_path)

    else:
      # Delete the compressed file
      os.remove(file_path)
      print("🗑️ Deleted compressed file:", file_path)




async def download_file(message):
    try:
        # Download the file
        file_path = os.path.join(DOWNLOAD_PATH, message.file.name)
        await client.download_media(message.media.document, file=file_path)
        print("⬇️ File downloaded:", file_path)
    except TypeError as e:
        print("❌ Unable to get file name while downloading photo:", e, " - ", message)
    except Exception as e:
        print("❌ Error while downloading file:", e, " - ", message)
    else:
        # Check if the downloaded file is a compressed file
        if is_compressed_file(file_path):
            extract_folder = os.path.join(DOWNLOAD_PATH, os.path.splitext(message.file.name)[0])
            extract_compressed_file(file_path, extract_folder)


async def download_photo(message):
    try:
        # Download the photo
        file_path = os.path.join(DOWNLOAD_PATH, f"{message.id}.jpg")
        await client.download_media(message.media.photo, file=file_path)
        print("⬇️ Photo downloaded:", file_path)
    except TypeError as e:
        print("❌ Unable to get file name while downloading photo:", e, " - ", message)
    except Exception as e:
        print("❌ Error while downloading photo:", e, " - ", message)



async def get_channel_id(channel_info):
  try:
    # If a channel id was given instead of a channel link, it must be cast to PeerChannel type
    if isinstance(channel_info, int):
      channel_info = PeerChannel(channel_info)

    result = await client.get_entity(channel_info)
  except Exception as e:
    print("Channel not found - ", channel_info, ' : ', e)
    return
  else:
    # Print the channel Title & ID
    print("Channel: ", result.title, ", Channel ID:", result.id)

    # return the channel ID
    return result.id




async def event_listener():
  global client
  async with TelegramClient(SESSION_NAME, API_ID, API_HASH) as client:
    print("Listening to channels below:")
    # Get the desired channels entity
    listening_channels = []
    for channel in channels_to_listen:
      result = await get_channel_id(channel)
      if result and result not in listening_channels:
          listening_channels.append(result)

    # Add event handler to filter and forward file messages
    client.add_event_handler(forward_to_saved_messages, event=events.NewMessage(chats=listening_channels))

    # Start the event listener
    await client.run_until_disconnected()





#Create download folder
try:
  os.mkdir(DOWNLOAD_PATH)
except FileExistsError:
  print("Download folder already exist!")
else:
  print("Download folder created!")



try:
  '''
  loop = asyncio.get_event_loop()
  loop.run_until_complete(event_listener())
  '''
  asyncio.run(event_listener())
except RuntimeError:
  await event_listener()